# Machina Labs
## Homework/Problem Solution

Thank you for the opportunity to interview at Machina Labs. This was a really interesting dataset to work on and, in particular, it was instructive to model real world data.

I've tried to structure this jupyter notebook as cleanly as I could so that it is easy to follow different approaches I've tried. Also, for every model I trained I have stored a copy (as a pickle file) in the director `models`. This will allow you to see results without the need to personally re-train the models. For some methods (e.g. Neural Networks), the training time is quite long (especially without a dedicated GPU) so loading the model will be quick.

I've provided an environment.yml file with the necessary python packages. If you're using conda, you should be able to easily create a new environment with

```bash
conda env create -p machina_hw -f environment.yml
```

I have also moved the 3 data files (`Test1.csv`, `Test2.csv`, `Test4.csv`) to a dedicated data directory (i.e. `data`).